# LSTM con embeddings

Modelo básico con los embeddings de fasttext

In [1]:
import pandas as pd
import csv

df_dev = pd.read_table("../../../data/es/dev_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_train = pd.read_table("../../../data/es/train_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)

text_train, y_train = df_train["text"], df_train["HS"]
text_dev, y_dev = df_dev["text"], df_dev["HS"]

print("Instancias de entrenamiento: {}".format(len(df_train)))
print("Instancias de desarrollo: {}".format(len(df_dev)))


Instancias de entrenamiento: 4500
Instancias de desarrollo: 500


Tengo que hacer dos cosas:

- Primero, convertir los tweets a secuencias de texto
- Luego, paddear las secuencias a cierta longitud (Keras necesita esto para poder paralelizar cálculo)

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words = 200000

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(text_train)

X_train = tokenizer.texts_to_sequences(text_train)
X_dev = tokenizer.texts_to_sequences(text_dev)

max_length = 30

X_train = pad_sequences(X_train, max_length)
X_dev = pad_sequences(X_dev, max_length)

Using TensorFlow backend.


Carguemos embeddings

In [3]:
import os
from glob import glob

path_to_embeddings = os.path.expanduser("/home/jmperez/WordVectors/")

print("Available embeddings: ", glob(os.path.join(path_to_embeddings, "*.vec")))

Available embeddings:  ['/home/jmperez/WordVectors/UBA_w5_200.vec', '/home/jmperez/WordVectors/wiki.es.vec', '/home/jmperez/WordVectors/UBA_w5_300.vec']


# Twitter Embeddings

In [4]:
import numpy as np

word_to_vec = {}

with open(os.path.join(path_to_embeddings, "UBA_w5_300.vec")) as f:
    for line in f:
        values = line.split()
        word = values[0]
        try:
            vec = np.asarray(values[1:], dtype="float32")
        except:
            print(("*" * 80  + "\n")*3)
            print("Problema con la sig línea:")
            print(values[:10])
            word = values[1]
            vec = np.asarray(values[2:], dtype="float32")
        word_to_vec[word] = vec
        
embedding_size = len(word_to_vec["hola"])

********************************************************************************
********************************************************************************
********************************************************************************

Problema con la sig línea:
['.', '.', '-0.22232', '0.0052569', '0.47066', '0.13836', '0.15991', '0.19504', '0.00067885', '0.020299']
********************************************************************************
********************************************************************************
********************************************************************************

Problema con la sig línea:
['.', '...', '-0.11666', '-0.083768', '0.028919', '0.29973', '0.21017', '0.27808', '0.063251', '0.090223']
********************************************************************************
********************************************************************************
********************************************************************************

P

In [5]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((num_words, embedding_size))
for word, i in tokenizer.word_index.items():
    embedding_vector = word_to_vec.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [6]:
embedding_matrix.shape

(200000, 300)

In [10]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, Conv1D, Flatten
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.preprocessing import sequence


model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.5))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

optimizer_args = {
    "lr": 0.001,
    "decay": 0.01
}

model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])

In [11]:
model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=20, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/20
4500/4500 [==============================] - 8s 2ms/step - loss: 0.6365 - acc: 0.6493 - val_loss: 0.5870 - val_acc: 0.7100
Epoch 2/20
4500/4500 [==============================] - 7s 2ms/step - loss: 0.5319 - acc: 0.7464 - val_loss: 0.5333 - val_acc: 0.7700
Epoch 3/20
4500/4500 [==============================] - 7s 2ms/step - loss: 0.4895 - acc: 0.7700 - val_loss: 0.5290 - val_acc: 0.7520
Epoch 4/20
4500/4500 [==============================] - 7s 2ms/step - loss: 0.4564 - acc: 0.7898 - val_loss: 0.5037 - val_acc: 0.7740
Epoch 5/20
4500/4500 [==============================] - 7s 2ms/step - loss: 0.4303 - acc: 0.8036 - val_loss: 0.5117 - val_acc: 0.7680
Epoch 6/20
4500/4500 [==============================] - 7s 2ms/step - loss: 0.4044 - acc: 0.8169 - val_loss: 0.4935 - val_acc: 0.7840
Epoch 7/20
4500/4500 [==============================] - 7s 2ms/step - loss: 0.3912 - acc: 0.8267 - val_loss: 0.4900 - val_acc: 0.7980
Epoch 8/20
4500

In [12]:
from hate.utils import print_evaluation

print_evaluation(model, X_dev, y_dev.values)

500/500 [==============================] - 0s 637us/step
Loss        : 0.5210
Accuracy    : 0.7900
Precision   : 0.7448
Recall      : 0.8018
F1          : 0.7722


## Bidirectional LSTM

In [13]:
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, Conv1D, Flatten, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

optimizer_args = {
    "lr": 0.001,
    "decay": 0.01
}

model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])

In [14]:
model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=20, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/20
4500/4500 [==============================] - 14s 3ms/step - loss: 0.6167 - acc: 0.6598 - val_loss: 0.6379 - val_acc: 0.6540
Epoch 2/20
4500/4500 [==============================] - 12s 3ms/step - loss: 0.5100 - acc: 0.7529 - val_loss: 0.6359 - val_acc: 0.6320
Epoch 3/20
4500/4500 [==============================] - 12s 3ms/step - loss: 0.4582 - acc: 0.7882 - val_loss: 0.5076 - val_acc: 0.7520
Epoch 4/20
4500/4500 [==============================] - 12s 3ms/step - loss: 0.4246 - acc: 0.8071 - val_loss: 0.5161 - val_acc: 0.7440
Epoch 5/20
4500/4500 [==============================] - 12s 3ms/step - loss: 0.3954 - acc: 0.8198 - val_loss: 0.4887 - val_acc: 0.7520
Epoch 6/20
4500/4500 [==============================] - 12s 3ms/step - loss: 0.3765 - acc: 0.8313 - val_loss: 0.4759 - val_acc: 0.7820
Epoch 7/20
4500/4500 [==============================] - 12s 3ms/step - loss: 0.3531 - acc: 0.8438 - val_loss: 0.4841 - val_acc: 0.7700
Epoch 8/

In [15]:
from hate.utils import print_evaluation
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print_evaluation(model, X_dev, y_dev.values)

500/500 [==============================] - 1s 1ms/step
Loss        : 0.5671
Accuracy    : 0.7840
Precision   : 0.7317
Recall      : 0.8108
F1          : 0.7692


## Probemos GRU + Conv

In [16]:
from keras.layers import GRU

model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(GRU(100, return_sequences=True))
model.add(Dropout(0.5))
model.add(Conv1D(64, 3, activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(1, activation='sigmoid'))

optimizer_args = {
    "lr": 0.001,
    "decay": 0.01
}

model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])

model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=20, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/20
4500/4500 [==============================] - 7s 2ms/step - loss: 0.6128 - acc: 0.6702 - val_loss: 0.5938 - val_acc: 0.7060
Epoch 2/20
4500/4500 [==============================] - 6s 1ms/step - loss: 0.5074 - acc: 0.7520 - val_loss: 0.5858 - val_acc: 0.7060
Epoch 3/20
4500/4500 [==============================] - 6s 1ms/step - loss: 0.4611 - acc: 0.7880 - val_loss: 0.5462 - val_acc: 0.7300
Epoch 4/20
4500/4500 [==============================] - 6s 1ms/step - loss: 0.4329 - acc: 0.8027 - val_loss: 0.5389 - val_acc: 0.7380
Epoch 5/20
4500/4500 [==============================] - 6s 1ms/step - loss: 0.4090 - acc: 0.8120 - val_loss: 0.5312 - val_acc: 0.7580
Epoch 6/20
4500/4500 [==============================] - 6s 1ms/step - loss: 0.3922 - acc: 0.8231 - val_loss: 0.5357 - val_acc: 0.7560
Epoch 7/20
4500/4500 [==============================] - 6s 1ms/step - loss: 0.3795 - acc: 0.8300 - val_loss: 0.5340 - val_acc: 0.7520
Epoch 8/20
4500

In [17]:
from hate.utils import print_evaluation

print_evaluation(model, X_dev, y_dev.values)

500/500 [==============================] - 0s 494us/step
Loss        : 0.5585
Accuracy    : 0.7560
Precision   : 0.7137
Recall      : 0.7523
F1          : 0.7325


# Conv + LSTM

In [23]:
from keras.layers import GRU, MaxPooling1D

model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0.55))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.35))
model.add(Dense(1, activation='sigmoid'))
optimizer_args = {
    "lr": 0.001,
    "decay": 0.02
}

model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=20, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/20
4500/4500 [==============================] - 7s 1ms/step - loss: 0.6444 - acc: 0.6451 - val_loss: 0.6352 - val_acc: 0.6720
Epoch 2/20
4500/4500 [==============================] - 4s 983us/step - loss: 0.5627 - acc: 0.7253 - val_loss: 0.5556 - val_acc: 0.7100
Epoch 3/20
4500/4500 [==============================] - 4s 957us/step - loss: 0.5096 - acc: 0.7631 - val_loss: 0.5343 - val_acc: 0.7400
Epoch 4/20
4500/4500 [==============================] - 4s 964us/step - loss: 0.4762 - acc: 0.7858 - val_loss: 0.5265 - val_acc: 0.7460
Epoch 5/20
4500/4500 [==============================] - 4s 990us/step - loss: 0.4561 - acc: 0.7982 - val_loss: 0.5194 - val_acc: 0.7560
Epoch 6/20
4500/4500 [==============================] - 4s 977us/step - loss: 0.4347 - acc: 0.8073 - val_loss: 0.5121 - val_acc: 0.7480
Epoch 7/20
4500/4500 [==============================] - 4s 987us/step - loss: 0.4258 - acc: 0.8191 - val_loss: 0.5142 - val_acc: 0.7540
Epo

In [24]:
from hate.utils import print_evaluation

print_evaluation(model, X_dev, y_dev.values)

500/500 [==============================] - 0s 352us/step
Loss        : 0.5281
Accuracy    : 0.7940
Precision   : 0.7820
Recall      : 0.7432
F1          : 0.7621
